In [7]:
import os
import shutil

dataset_dir = "D:/dataset"
classes_list = os.listdir(dataset_dir)

base_dir = "D:/splitted"
os.makedirs(base_dir,exist_ok=True)

train_dir = os.path.join(base_dir,"train")
os.makedirs(train_dir,exist_ok=True)
validation_dir = os.path.join(base_dir, "validation")
os.makedirs(validation_dir,exist_ok=True)
test_dir = os.path.join(base_dir, "test")
os.makedirs(test_dir,exist_ok=True)

classes_list

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Cherry___healthy',
 'Cherry___Powdery_mildew',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn___Common_rust',
 'Corn___healthy',
 'Corn___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [8]:
for cls in classes_list:
    os.makedirs(os.path.join(train_dir,cls),exist_ok=True)
    os.makedirs(os.path.join(validation_dir,cls),exist_ok=True)
    os.makedirs(os.path.join(test_dir,cls),exist_ok=True)
    

In [ ]:
import math

for cls in classes_list:
    path = os.path.join(dataset_dir,cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames)*0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)


    train_fnames = fnames[:train_size]
    print()

